In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/aaaaaaa/House Price.csv


In [5]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import ipywidgets as widgets
from IPython.display import display

# Load data
data = pd.read_csv('/kaggle/input/aaaaaaa/House Price.csv')

# Preprocessing
binary_cols = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 
              'airconditioning', 'prefarea']
for col in binary_cols:
    data[col] = data[col].map({'yes': 1, 'no': 0})
    
le = LabelEncoder()
data['furnishingstatus'] = le.fit_transform(data['furnishingstatus'])

# Train model
X = data.drop('price', axis=1)
y = data['price']
model = DecisionTreeRegressor(random_state=42)
model.fit(X, y)

# Create widgets with corrected Dropdown syntax
inputs = {
    'area': widgets.IntSlider(value=5000, min=1000, max=20000, description='Area (sq ft):'),
    'bedrooms': widgets.IntSlider(value=3, min=1, max=6, description='Bedrooms:'),
    'bathrooms': widgets.IntSlider(value=2, min=1, max=4, description='Bathrooms:'),
    'stories': widgets.IntSlider(value=2, min=1, max=4, description='Stories:'),
    'parking': widgets.IntSlider(value=1, min=0, max=3, description='Parking:'),
    'mainroad': widgets.Dropdown(options=['Yes', 'No'], value='Yes', description='Main Road:'),
    'guestroom': widgets.Dropdown(options=['Yes', 'No'], value='No', description='Guest Room:'),
    'basement': widgets.Dropdown(options=['Yes', 'No'], value='No', description='Basement:'),
    'hotwaterheating': widgets.Dropdown(options=['Yes', 'No'], value='No', description='Hot Water:'),
    'airconditioning': widgets.Dropdown(options=['Yes', 'No'], value='Yes', description='AC:'),
    'prefarea': widgets.Dropdown(options=['Yes', 'No'], value='Yes', description='Pref Area:'),
    'furnishingstatus': widgets.Dropdown(
        options=['Furnished', 'Semi-Furnished', 'Unfurnished'],
        value='Furnished',
        description='Furnishing:'
    )
}

predict_btn = widgets.Button(description="Predict Price")
output = widgets.Output()

def on_predict(b):
    input_data = pd.DataFrame({
        'area': [inputs['area'].value],
        'bedrooms': [inputs['bedrooms'].value],
        'bathrooms': [inputs['bathrooms'].value],
        'stories': [inputs['stories'].value],
        'mainroad': [1 if inputs['mainroad'].value == 'Yes' else 0],
        'guestroom': [1 if inputs['guestroom'].value == 'Yes' else 0],
        'basement': [1 if inputs['basement'].value == 'Yes' else 0],
        'hotwaterheating': [1 if inputs['hotwaterheating'].value == 'Yes' else 0],
        'airconditioning': [1 if inputs['airconditioning'].value == 'Yes' else 0],
        'parking': [inputs['parking'].value],
        'prefarea': [1 if inputs['prefarea'].value == 'Yes' else 0],
        'furnishingstatus': [
            0 if inputs['furnishingstatus'].value == 'Furnished' else
            1 if inputs['furnishingstatus'].value == 'Semi-Furnished' else 2
        ]
    })
    
    prediction = model.predict(input_data)[0]
    with output:
        output.clear_output()
        print(f"Predicted Price: ₹{prediction:,.2f}")
        print(f"\nMarket Statistics:")
        print(f"Average: ₹{y.mean():,.2f}")
        print(f"Minimum: ₹{y.min():,.2f}")
        print(f"Maximum: ₹{y.max():,.2f}")

predict_btn.on_click(on_predict)

# Display the app
display(widgets.VBox([
    widgets.HBox([inputs['area'], inputs['bedrooms'], inputs['bathrooms']]),
    widgets.HBox([inputs['stories'], inputs['parking'], inputs['mainroad']]),
    widgets.HBox([inputs['guestroom'], inputs['basement'], inputs['hotwaterheating']]),
    widgets.HBox([inputs['airconditioning'], inputs['prefarea'], inputs['furnishingstatus']]),
    predict_btn,
    output
]))